In [1]:
from utils.elpv_reader import load_dataset
import numpy as np
import matplotlib as plt
import cv2
import os

# scikit-learn imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit

# tensorflow imports
import tensorflow
from tensorflow import keras
from keras.applications import VGG19
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D
from keras import Input
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras import layers

# from PIL import Image
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
images, probs, types = load_dataset()
print(images.shape)
print(probs.shape)
print(types.shape)

(2624, 300, 300)
(2624,)
(2624,)


In [3]:
# imagesFlipV = np.zeros(images.shape)
# probsFlipV = probs.copy()
# typesFlipV = types.copy()

# for i in range(0, len(images)):
#     imagesFlipV[i] = cv2.flip(images[i], 0)

# images = np.append(images, imagesFlipV, axis=0)
# probs = np.append(probs, probsFlipV)
# types = np.append(types, typesFlipV)

In [4]:
# index = 132

# plt.title("original")
# plt.imshow(images[index], cmap='gray')
# plt.show()


# norm_img1 = cv2.normalize(images[index], None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
# norm_img1 = (255*norm_img1).astype(np.uint8)
# plt.title("normalised")
# plt.imshow(norm_img1, cmap='gray')
# plt.show()


# kernelLaplacian = np.array([[1, 1, 1],
#                            [1, -8, 1],
#                            [1, 1, 1]])
# edging = cv2.filter2D(src=norm_img1, ddepth=-1, kernel=kernelLaplacian)
# plt.title("laplacian")
# plt.imshow(edging, cmap='gray', vmin=0, vmax=255)
# plt.show()


# t_lower = 25
# t_upper = 120
# cannyEdges = cv2.Canny(norm_img1, t_lower, t_upper)
# plt.title("canny")
# plt.imshow(cannyEdges, cmap='gray')
# plt.show()

# retOtsu,thOtsu = cv2.threshold(images[index],0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
# plt.title("otsu")
# plt.imshow(thOtsu, cmap='gray')
# plt.show()

In [5]:
# OTSU IMAGE SEGMENTATION HUEHUEHUE
# uncomment below code for otsu segmentation
# for i in range(0, len(images)):
#     retOtsu,thOtsu = cv2.threshold(images[i],0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
#     images[i] = thOtsu

# kernelLaplacian = np.array([[1, 1, 1],
#                             [1, -8, 1],
#                             [1, 1, 1]])

# for i in range(0, len(images)):
#     norm_img = cv2.normalize(images[i], None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
#     norm_img = (255*norm_img).astype(np.uint8)
#     images[i] = norm_img
#     # edging = cv2.filter2D(src=norm_img, ddepth=-1, kernel=kernelLaplacian)
#     # images[i] = edging


def isPoly(formatted_monopoly_data):
    res = np.zeros(len(formatted_monopoly_data))
    for i in range(0, len(formatted_monopoly_data)):
        res[i] = int(formatted_monopoly_data[i][1])
    
    return res

def getArgMax(probabilities):
    res = np.zeros(len(probabilities))
    for i in range(0, len(probabilities)):
        res[i] = np.argmax(probabilities[i])

    return res


In [6]:
# MAKE BIG TRAINING AND TESTING SET
images_3 = np.dstack([images] * 3)
images_3 = np.reshape(images_3, (-1, 300, 300, 3))

probs_formatted = probs
probs_label_encoder = LabelEncoder()
probs_integer = probs_label_encoder.fit_transform(probs_formatted)
probs_formatted = keras.utils.to_categorical(probs_integer)

x_train, x_test, y_train, y_test = train_test_split(images_3, probs_formatted, test_size=0.25, random_state = 420)

In [7]:
# do the same with VGG, but for mono and poly images
vgg19_base = keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=(300, 300, 3))
vgg19_base.trainable = False

vgg19_model = keras.models.Sequential([
  vgg19_base,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Flatten(input_shape=vgg19_base.output_shape[1:]),
  keras.layers.Dense(4096, activation='relu', kernel_initializer='he_normal'),
  keras.layers.Dense(2048, activation='relu', kernel_initializer='he_normal'),
  keras.layers.Dense(len(np.unique(probs)), activation='softmax')
])

vgg19_model.compile(optimizer=keras.optimizers.Adam(learning_rate = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon=1e-08),
              loss="categorical_crossentropy",
              metrics=['accuracy'])

es = keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = 3)

history_mono = vgg19_model.fit(x_train, y_train, epochs=10, validation_split = 0.2, callbacks=[es], batch_size=16)

PATH = 'models/vgg19_all_model.keras'
vgg19_model.save(PATH)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Train on 1574 samples, validate on 394 samples
Epoch 1/10
1574/1574 [==============================] - 14s 9ms/sample - loss: 5.7919 - acc: 0.6182 - val_loss: 1.1096 - val_acc: 0.6472
Epoch 2/10
1574/1574 [==============================] - 14s 9ms/sample - loss: 0.9652 - acc: 0.6900 - val_loss: 0.8861 - val_acc: 0.6624
Epoch 3/10
1574/1574 [==============================] - 14s 9ms/sample - loss: 0.6339 - acc: 0.7548 - val_loss: 0.8752 - val_acc: 0.7259
Epoch 4/10
1574/1574 [==============================] - 14s 9ms/sample - loss: 0.6113 - acc: 0.7706 - val_loss: 0.8119 - val_acc: 0.7284
Epoch 5/10
1574/1574 [==============================] - 14s 9ms/sample - loss: 0.5620 - acc: 0.7891 - val_loss: 0.8450 - val_acc: 0.7563
Epoch 6/10
1574/1574 [==============================] - 14s 9ms/sample - loss: 0.5287 - acc: 0.7967 - val_loss: 0.7706 - val_acc: 0.7259
Epoch 7/10
1574/1574 [==============================] - 14s 9ms/sample - loss: 0.5272 - acc: 0.8062 - val_loss: 0.8029 - val_acc: 0

In [9]:
# score = vgg19_model.evaluate(x_test, y_test)
result = vgg19_model.predict(x_test)
# print(dict(zip(vgg19_model.metrics_names, score)))
# print(vgg19_model.metrics_names)

def getArgMax(probabilities):
    res = np.zeros(len(probabilities))
    for i in range(0, len(probabilities)):
        res[i] = np.argmax(probabilities[i])

    return res


predicted_int = getArgMax(result)
y_test_int = getArgMax(y_test)

from sklearn.metrics import classification_report
print(classification_report(predicted_int, y_test_int))

              precision    recall  f1-score   support

         0.0       0.82      0.79      0.80       396
         1.0       0.43      0.30      0.35       101
         2.0       0.00      0.00      0.00         1
         3.0       0.71      0.82      0.76       158

    accuracy                           0.72       656
   macro avg       0.49      0.47      0.48       656
weighted avg       0.74      0.72      0.72       656

